In [ ]:
#Importing required libraries

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [ ]:
# Importing predefined weights

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=True)


In [ ]:
# Lock the dataset to prevent unwanted changes.

base_model.trainable = False

In [ ]:
# Building the model

inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)

outputs = keras.layers.Dense(6, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
# Summary of the built model

model.summary()

In [ ]:
# Compiling the model

model.compile(
    loss=keras.losses.BinaryCrossentropy(
    from_logits=True), 
    metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
datagen = ImageDataGenerator(
    samplewise_center=True,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False)

train_it = datagen.flow_from_directory('data/fruits/train/',
                                       target_size=(224, 224),
                                       color_mode='rgb',
                                       class_mode="categorical")

valid_it = datagen.flow_from_directory('data/fruits/valid/',
                                       target_size=(224, 224),
                                       color_mode='rgb',
                                       class_mode="categorical")



In [ ]:
# Fitting the dataset in model to be compiled

model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=5)

In [ ]:
# Unlocking the dataset to train

base_model.trainable = True

In [ ]:
# Compiling the model again

model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=.00001),  # Very low learning rate to not take big jumps
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()])


# Training

model.fit(
    train_it,
    validation_data=valid_it,
    steps_per_epoch=train_it.samples/train_it.batch_size,
    validation_steps=valid_it.samples/valid_it.batch_size,
    epochs=5)

In [ ]:
# Evaluating the model

model.evaluate(
    valid_it, 
    steps=valid_it.samples/valid_it.batch_size)